In [4]:
import spacy

nlp = spacy.load("en_core_web_sm")


def process_query(input_query):
    doc = nlp(input_query)
    output = {'select_column': None, 'from_table': None, 'where': []}

    # Identify entities and conditions in the query
    for token in doc:
        if token.dep_ == 'nsubj':
            output['select_column'] = token.text
        elif token.dep_ == 'ROOT':
            output['from_table'] = token.text
        elif token.dep_ == 'attr' and token.head.dep_ == 'prep':
            output['select_column'] = token.head.text
        elif token.dep_ == 'pobj' and token.head.dep_ == 'prep':
            output['from_table'] = token.text
        elif token.dep_ == 'nsubjpass' and token.head.dep_ == 'prep':
            output['from_table'] = token.text

    # Extract conditions
    for ent in doc.ents:
        if ent.label_ == 'GPE' or ent.label_ == 'PERSON':
            output['where'].append({'column': 'ship_name', 'relation': '=', 'value': ent.text})
        elif ent.label_ == 'CARDINAL':
            output['where'].append({'column': 'grid_ram', 'relation': '>', 'value': int(ent.text)})
        elif ent.text == 'blockage':
            output['where'].append({'column': 'is_in_blockage', 'relation': '=', 'value': 1})
        elif ent.text == 'signal':
            output['where'].append({'column': 'system_signal', 'relation': '=', 'value': 1})
        elif ent.text == 'SNR':
            output['where'].append({'column': 'system_snr', 'relation': '=', 'value': int(doc[doc.index(ent) + 1].text)})
        elif ent.text == 'uptime':
            output['where'].append({'column': 'grid_uptime', 'relation': '>', 'value': int(doc[doc.index(ent) + 1].text) * 60})

    return output


# Test cases
queries = [
    "Which ships have a grid with more than 50 gb ram?",
    "Which system is in the blockage on ship umbra?",
    "Which grid in nebula has more than 60gb ram?",
    "Name of the ship which is in blockage and has the first letter as ‘g’?",
    "What are the grid versions currently active on the ship eta?",
    "What are the signal strength on the ship ‘mu’?",
    "When was grid hub1_LD5 updated on the ship gamma?",
    "Which ships have a sound to noise ratio of 50?",
    "Which ships have a grid consuming less than 20 gb memory but has an uptime of more than 6 hours?",
    "Snr of SCPC_1 system in theta ship ?"
]

for query in queries:
    output = process_query(query)
    print(f"INPUT: {query}")
    print("OUTPUT:", output)
    print()


ValueError: invalid literal for int() with base 10: 'more than 50'

In [29]:
from nltk import pos_tag
from nltk.tokenize import word_tokenize

def pos_tagging(query):
    tokens = word_tokenize(query)
    tagged = pos_tag(tokens)
    noun = []
    for i in range(len(tagged)):
        if tagged[i][1] == 'NN':
            noun.append(tagged[i][0])
    return tagged

# Example
query = "Which system is in the blockage on ship umbra?"
query2 = "Name of the ship which is in blockage and has the first letter as ‘g’?"
query3 = "Which ships have a grid with more than 50 gb ram?"
query4 = "Which grid in nebula has more than 60gb ram?"
query5 = "Name of the ship which is in blockage and has the first letter as ‘g’?"

query6 = "What are the signal strength on the ship ‘mu’?"
query7 = "When was grid hub1_LD5 updated on the ship gamma?"
query8 = "Which ships have a sound to noise ratio of 50?"

pos_tags = pos_tagging(query)
pos_tags2 = pos_tagging(query2)
pos_tags3 = pos_tagging(query3)
pos_tags4 = pos_tagging(query4)
pos_tags5 = pos_tagging(query5)
pos_tags5 = pos_tagging(query6)
pos_tags6 = pos_tagging(query7)
pos_tags7 = pos_tagging(query8)
print(pos_tags,pos_tags[0][1])
print(pos_tags2)
print(pos_tags3)
print(pos_tags4)
print(pos_tags5)
print(pos_tags6)
print(pos_tags7)


[('Which', 'JJ'), ('system', 'NN'), ('is', 'VBZ'), ('in', 'IN'), ('the', 'DT'), ('blockage', 'NN'), ('on', 'IN'), ('ship', 'NN'), ('umbra', 'NN'), ('?', '.')] JJ
[('Name', 'NN'), ('of', 'IN'), ('the', 'DT'), ('ship', 'NN'), ('which', 'WDT'), ('is', 'VBZ'), ('in', 'IN'), ('blockage', 'NN'), ('and', 'CC'), ('has', 'VBZ'), ('the', 'DT'), ('first', 'JJ'), ('letter', 'NN'), ('as', 'IN'), ('‘', 'NN'), ('g', 'NN'), ('’', 'NN'), ('?', '.')]
[('Which', 'JJ'), ('ships', 'NNS'), ('have', 'VBP'), ('a', 'DT'), ('grid', 'NN'), ('with', 'IN'), ('more', 'JJR'), ('than', 'IN'), ('50', 'CD'), ('gb', 'NN'), ('ram', 'NN'), ('?', '.')]
[('Which', 'WDT'), ('grid', 'NN'), ('in', 'IN'), ('nebula', 'NN'), ('has', 'VBZ'), ('more', 'JJR'), ('than', 'IN'), ('60gb', 'CD'), ('ram', 'NN'), ('?', '.')]
[('What', 'WP'), ('are', 'VBP'), ('the', 'DT'), ('signal', 'JJ'), ('strength', 'NN'), ('on', 'IN'), ('the', 'DT'), ('ship', 'NN'), ('‘', 'NNP'), ('mu', 'NN'), ('’', 'NN'), ('?', '.')]
[('When', 'WRB'), ('was', 'VBD'), 

In [11]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

def parse_query(query):
    tokens = word_tokenize(query.lower())  # Tokenize and convert to lowercase
    stop_words = set(stopwords.words('english'))  # Remove stopwords
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]

    # Define relations and operators
    relations = ['=', '>', '<', 'like']
    operators = ['and', 'or']

    # Initialize variables
    select_column = None
    from_table = None
    where_conditions = []

    # Parse the query
    for i, token in enumerate(filtered_tokens):
        if token == 'ship':
            from_table = 'rf_stats'
        elif token == 'grid':
            from_table = 'grid_details'
        elif token in relations:
            if filtered_tokens[i - 1] == 'ship' or filtered_tokens[i - 1] == 'grid':
                condition = {'column': filtered_tokens[i - 1] + '_name',
                             'relation': token}
                if token == 'like':
                    condition['value'] = filtered_tokens[i + 1] + '%'
                else:
                    condition['value'] = filtered_tokens[i + 1]
                where_conditions.append(condition)
        elif token in operators:
            continue
        else:
            select_column = token

    return {'select_column': select_column, 'from_table': from_table, 'where': where_conditions}

queries = [
    "Which ships have a grid with more than 50 gb ram?",
    "Which system is in the blockage on ship umbra?",
    "Which grid in nebula has more than 60gb ram?",
    "Name of the ship which is in blockage and has the first letter as ‘g’?",
    "What are the grid versions currently active on the ship eta?",
    "What are the signal strength on the ship ‘mu’?",
    "When was grid hub1_LD5 updated on the ship gamma?",
    "Which ships have a sound to noise ratio of 50?",
    "Which ships have a grid consuming less than 20 gb memory but has an uptime of more than 6 hours?",
    "Snr of SCPC_1 system in theta ship ?"
]

for query in queries:
    result = parse_query(query)
    print(f"Input: {query}")
    print("Output: ->", result)
    print()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...


Input: Which ships have a grid with more than 50 gb ram?
Output: -> {'select_column': 'ram', 'from_table': 'grid_details', 'where': []}

Input: Which system is in the blockage on ship umbra?
Output: -> {'select_column': 'umbra', 'from_table': 'rf_stats', 'where': []}

Input: Which grid in nebula has more than 60gb ram?
Output: -> {'select_column': 'ram', 'from_table': 'grid_details', 'where': []}

Input: Name of the ship which is in blockage and has the first letter as ‘g’?
Output: -> {'select_column': 'g', 'from_table': 'rf_stats', 'where': []}

Input: What are the grid versions currently active on the ship eta?
Output: -> {'select_column': 'eta', 'from_table': 'rf_stats', 'where': []}

Input: What are the signal strength on the ship ‘mu’?
Output: -> {'select_column': 'mu', 'from_table': 'rf_stats', 'where': []}

Input: When was grid hub1_LD5 updated on the ship gamma?
Output: -> {'select_column': 'gamma', 'from_table': 'rf_stats', 'where': []}

Input: Which ships have a sound to nois

[nltk_data]   Unzipping corpora\stopwords.zip.


In [20]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

# Define the datasets and column details
datasets = {
    'rf_stats': {
        'columns': ['ship_name', 'system', 'system_signal', 'is_in_blockage', 'system_snr']
    },
    'grid_details': {
        'columns': ['ship_name', 'grid_name', 'grid_version', 'grid_uptime', 'grid_time_since_version_change', 'grid_ram', 'grid_last_version']
    }
}

# Define the NLTK stopwords
stop_words = set(stopwords.words('english'))

def generate_query(input_text):
    tokens = word_tokenize(input_text)
    filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]
    print(filtered_tokens)
    select_column = ''
    from_table = ''
    where = []
    
    if 'ship' in filtered_tokens and 'name' in filtered_tokens:
        if 'grid' in filtered_tokens:
            select_column = 'ship_name'
            from_table = 'grid_details'
        else:
            select_column = 'system'
            from_table = 'rf_stats'
            
        ship_index = filtered_tokens.index('ship')
        ship_name = filtered_tokens[ship_index + 1]
        where.append({'column': 'ship_name', 'relation': '=', 'value': ship_name})
        
        if 'grid' in filtered_tokens:
            ram_index = filtered_tokens.index('ram')
            condition_value = filtered_tokens[ram_index - 1]
            where.append({'column': 'grid_ram', 'relation': '>', 'value': condition_value})
        
    # Add more conditions based on the input_text
    
    query = {
        'select_column': select_column,
        'from_table': from_table,
        'where': where
    }
    
    return query

# Test the function with the provided test cases
test_cases = [
    "Which ships have a grid with more than 50 gb ram?",
    "Which system is in the blockage on ship umbra?",
    "Which grid in nebula has more than 60gb ram?",
    "Name of the ship which is in blockage and has the first letter as ‘g’?",
    "What are the grid versions currently active on the ship eta?",
    "What are the signal strength on the ship ‘mu’?",
    "When was grid hub1_LD5 updated on the ship gamma?",
    "Which ships have a sound to noise ratio of 50?",
    "Which ships have a grid consuming less than 20 gb memory but has an uptime of more than 6 hours?",
    "Snr of SCPC_1 system in theta ship?"
]

for test_case in test_cases:
    print(generate_query(test_case))

['Which', 'ships', 'have', 'a', 'grid', 'with', 'more', 'than', '50', 'gb', 'ram', '?']
['ships', 'grid', '50', 'gb', 'ram']
{'select_column': '', 'from_table': '', 'where': []}
['Which', 'system', 'is', 'in', 'the', 'blockage', 'on', 'ship', 'umbra', '?']
['system', 'blockage', 'ship', 'umbra']
{'select_column': '', 'from_table': '', 'where': []}
['Which', 'grid', 'in', 'nebula', 'has', 'more', 'than', '60gb', 'ram', '?']
['grid', 'nebula', '60gb', 'ram']
{'select_column': '', 'from_table': '', 'where': []}
['Name', 'of', 'the', 'ship', 'which', 'is', 'in', 'blockage', 'and', 'has', 'the', 'first', 'letter', 'as', '‘', 'g', '’', '?']
['name', 'ship', 'blockage', 'first', 'letter', 'g']
{'select_column': 'system', 'from_table': 'rf_stats', 'where': [{'column': 'ship_name', 'relation': '=', 'value': 'blockage'}]}
['What', 'are', 'the', 'grid', 'versions', 'currently', 'active', 'on', 'the', 'ship', 'eta', '?']
['grid', 'versions', 'currently', 'active', 'ship', 'eta']
{'select_column': 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
